In [30]:
#include a write up

In [31]:
import sqlite3
import pandas as pd
import numpy as np

In [32]:
conn = sqlite3.connect('./Census.db')
curs = conn.cursor()
curs.execute("PRAGMA foreign_keys=ON") #you don't have to use this command since their isn't anything changed in the data

In [33]:
# You might want to use this...
def getAll(sql, *args):
    temp = curs.execute(sql, (args)).fetchall()
    cols = [desc[0] for desc in curs.description]
    result = pd.DataFrame(temp, columns=cols)
    return result

In [34]:
#0) How many counties are represented in this data?
def GetCountyCount(ByState = False):
    if ByState:
        # Return the results grouped by state
        # Accesses the state and county (summed) columns from the tLocation table
        # Groups by state to access the number of counties for each state
        sql = """SELECT state, count(county) as CountiesByState FROM tLocation GROUP BY state"""
    else:
        # Don't group by state, just return the total number of counties
        # Accesses the county (summed) column from the tLocation table
        sql = """SELECT count(county) as TotalCounties FROM tLocation """ 
    return getAll(sql)


In [35]:
GetCountyCount(True)

,state,CountiesByState
0,Alabama,67
1,Alaska,29
2,Arizona,15
3,Arkansas,75
4,California,58
5,Colorado,64
6,Connecticut,8
7,Delaware,3
8,District of Columbia,1
9,Florida,67


In [36]:
GetCountyCount(False)

,TotalCounties
0,3142


In [51]:
#1) Given a county and state, return the number of housing units, sorted by year
# Accesses the state, county, year, and housing_estimate columns from the tHousing table
# take in inputs from the variables county and state
# Order the years from oldest to newest 
def GetHousingByCounty(county, state):
    sql =  """SELECT state, year, county, housing_estimate as housingUnits
              FROM tHousing                                       
                    JOIN tLocation ON tLocation.ID = tHousing.ID
              WHERE county = ? AND state = ?
              ORDER BY year ASC
            """
    return getAll(sql, county, state) 

In [52]:
# Test your function with county = "James City County" and state = "Virginia"
GetHousingByCounty("James City County", "Virginia")

,state,year,county,housingUnits
0,Virginia,2010,James City County,30136
1,Virginia,2011,James City County,30596
2,Virginia,2012,James City County,30926
3,Virginia,2013,James City County,31360
4,Virginia,2014,James City County,31795
5,Virginia,2015,James City County,32459
6,Virginia,2016,James City County,32877
7,Virginia,2017,James City County,33293
8,Virginia,2018,James City County,33763


In [53]:
#2) Given a county and state, return the number of housing units as well as the unemployment rate, sorted by year
# Accesses the state, county, year, unemployment_estimate, and housing_estimate columns from the tHousing table
# groups the data by state, county and year to access the exact data and time 
# Order the years from oldest to newest 
def GetHousingAndUnemployment(county, state):
    sql =  """SELECT tEmployment.year,state, county, housing_estimate as housingUnits, 
                   tEmployment.unemployment_estimate as unemploymentRate
               FROM tHousing
                   Join tEmployment ON tEmployment.year = tHousing.year 
                   AND tEmployment.ID = tHousing.ID 
                   Join tLocation ON tHousing.ID = tLocation.ID
               WHERE tLocation.county = ? AND tLocation.state = ?
               GROUP BY state, county, tEmployment.year
               ORDER BY tEmployment.year ASC
           """
    return getAll(sql,county, state)

In [40]:
# Test your function with county = "Pasco County" and state = "Florida"
GetHousingAndUnemployment("Pasco County", "Florida")

,year,state,county,housingUnits,unemploymentRate
0,2015,Florida,Pasco County,237055,6.4
1,2016,Florida,Pasco County,239487,7.5
2,2017,Florida,Pasco County,243185,5.3


In [41]:
#3) Given a state, what is the total population, sorted by year? 
# Accesses the state, year, total_estimate (summed) columns from the tEmployment table
# groups the data by year
# Order the years from oldest to newest 
def GetTotalPopByState(state):
    sql =  """SELECT state, year, sum(total_estimate) as totalPopulation
               FROM tEmployment                                        
                   Join tLocation ON tEmployment.ID = tLocation.ID     
              WHERE state = ?
              GROUP BY year
              ORDER BY year ASC
           """
    return getAll(sql, state)

In [42]:
# Test your function with state = Alaska
GetTotalPopByState("Alaska")

,state,year,totalPopulation
0,Alaska,2015,386701
1,Alaska,2016,389876
2,Alaska,2017,387304


In [62]:
#4) Given a state, what is the ratio of (population / housing unit), grouped and sorted by year?
# Accesses the state, county, year, total_estimate(summed) unemployment_estimate, and housing_estimate (summed)
# columns from the tEmployment table
# Also finds the ratio of total_estimate over housing estimate (summed)
# takes in the year from the state variable
# groups the data year
# Order the years from oldest to newest 
def GetHousingPopulationRatio(state):
    sql =  """SELECT tHousing.year, state, sum(tEmployment.total_estimate) 
                  as total_population, sum(housing_estimate) as total_housing,
                  (sum(tEmployment.total_estimate*1.0)/sum(housing_estimate*1.0)) as totalOverHousing          
               FROM tEmployment                                                                          
                   Join tHousing ON tEmployment.year = tHousing.year                                      
                   AND tEmployment.ID = tHousing.ID 
                   Join tLocation ON tEmployment.ID = tLocation.ID                  
               WHERE state = ?
               GROUP BY tHousing.year
               ORDER BY tHousing.year ASC
           """
    return getAll(sql, state)

In [63]:
# Test your function with state = Arizona
GetHousingPopulationRatio("Arizona")

,year,state,total_population,total_housing,totalOverHousing
0,2015,Arizona,5262531,2851601,1.845465
1,2016,Arizona,5352389,2880811,1.857945
2,2017,Arizona,5439794,2913951,1.866810


In [64]:
#5) In 2017, what were the 10 highest and 10 lowest counties in terms of median income?
#pd.concat() used to join the 10 highest and 10 lowest in one table
# Accesses the state, county, year, median_estimate columns from the tIncome table
# Also finds the ratio of total_estimate over housing estimate (summed)
# takes in the year from the state variable
# income data grouped from each county by their year
# Order the median_income from highest to lowest
def GetHighLowMedianIncome(year, n_high, n_low):
    sql =  """SELECT year, county, median_estimate as medianIncome
                FROM tIncome
                  Join tLocation ON tIncome.ID = tLocation.ID                  
              WHERE year = ?
              GROUP BY year, county
              ORDER BY medianIncome DESC
           """
    temp = getAll(sql,year)
    return pd.concat([temp.head(n_high), temp.tail(n_low)])

In [65]:
GetHighLowMedianIncome(2017,10,10)

,year,county,medianIncome
0,2017,Loudoun County,129588
1,2017,Fairfax County,117515
2,2017,Falls Church city,114795
3,2017,Arlington County,112138
4,2017,Douglas County,111154
5,2017,Hunterdon County,110969
6,2017,Los Alamos County,110190
7,2017,Fairfax city,106870
8,2017,Santa Clara County,106761
9,2017,San Mateo County,105667


In [66]:
#6) In 2017, which states had the highest and lowest population/housing ratio, 
# and what was their median income and unemployment rate?
#pd.concat() used to join the 10 highest and 10 lowest in one table
# Accesses the state, county, year, toal_estiamte, and housing_estimate columns from the tIncome 
# and tEmployment tables
# Also finds the ratio of total_estimate over housing estimate (summed)
# takes in the year from the state variable
# income data grouped from each county by their year
# Order the median_income from highest to lowest
def GetExtremeHousingRatios(year, n_high, n_low):
    sql =  """SELECT tIncome.year, state, (sum(tEmployment.total_estimate*1.0)/sum(tHousing.housing_estimate*1.0)) 
                  as medianRatio, tIncome.median_estimate, tEmployment.unemployment_estimate 
              FROM tIncome
                Join tHousing ON tHousing.year = tIncome.year 
                AND tEmployment.ID = tHousing.ID 
                Join tLocation ON tEmployment.ID = tLocation.ID 
                
                Join tEmployment ON tEmployment.year = tIncome.year 
                AND tEmployment.ID = tIncome.ID 

              WHERE tIncome.year = ?
              GROUP BY state, tIncome.year
              ORDER BY medianRatio DESC
          """
    temp = getAll(sql,year)
    return pd.concat([temp.head(n_high), temp.tail(n_low)])

In [67]:
GetExtremeHousingRatios(2017,5,5)

,year,state,medianRatio,median_estimate,unemployment_estimate
0,2017,California,2.234614,85743,4.7
1,2017,Utah,2.194946,53812,2.0
2,2017,Hawaii,2.125119,56395,5.5
3,2017,Maryland,2.028984,42771,8.1
4,2017,Texas,2.014596,46472,7.6
46,2017,West Virginia,1.744531,59480,6.5
47,2017,Montana,1.744459,46827,4.0
48,2017,Alabama,1.723467,52562,5.2
49,2017,North Dakota,1.721255,67308,1.5
50,2017,Maine,1.633275,49538,5.2
